## 1. 컨트랙 개발

In [96]:
%%writefile src/bank.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract bank {
    address owner;
    uint balance;
    
    event PrintLog(address from, uint amount);
    //event PrintLog(address from, address to, uint amount );
    
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }

    receive() external payable {
        uint amount = msg.value;
        balance += amount;
        emit PrintLog(msg.sender,msg.value);
    }

    function forwardTo(address payable _receiver) public payable {
        _receiver.transfer(msg.value);
        balance -= msg.value;
    }

    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }

    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }

    function withdrawAll() public {
        balance -= address(this).balance;
        msg.sender.transfer(address(this).balance); 
    }
}

Overwriting src/bank.sol


## 2. 컴파일

In [97]:
!solc --gas --bin --abi src/Bank.sol


======= src/Bank.sol:bank =======
Gas estimation:
construction:
   26063 + 134800 = 160863
external:
   deposit(uint256):	21103
   forwardTo(address):	infinite
   getBalance():	1027
   withdrawAll():	infinite
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006001819055506102a2806100686000396000f3fe6080604052600436106100435760003560e01c806312065fe0146100d057806327d8ad8814610102578063853828b614610146578063b6b55f251461015d576100cb565b366100cb576000349050806001600082825401925050819055507fac247564f70be8d5e06a6068acf596e7bbca33994b256697b38d49095d4f38053334604051808373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018281526020019250505060405180910390a150005b600080fd5b3480156100dc57600080fd5b506100e561018b565b604051808381526020018281526020019250505060405180910390f35b6101446004803603602081101561011857600080fd5b810190808035

In [98]:
!solc --optimize --combined-json abi,bin src/Bank.sol > src/Bank.json

## 3. 배포

In [99]:
%%writefile src/BankDeploy1.js
var Web3 = require('web3');
var _abiBinJson = require('./Bank.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8335"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/BankV3.sol:BankV3']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/BankDeploy1.js


In [100]:
!node src/BankDeploy1.js

- contract name:  [ 'src/Bank.sol:bank' ]
Deploying the contract from 0x309DD201450E258AE229E3C47D9c8C91a70F0123
hash: 0xe9c6bb45d82ccdfd0b682b4121b5a3787e9a3c4eff603edc1ff0568c83bc82b2
---> The contract deployed to: 0x32D192b0b16505aB0188748cf427a4d17Fbc15B6


## 4. 사용

In [101]:
%%writefile src/BankUse.js
var Web3=require('web3');
var _abiBinJson = require('./Bank.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8335"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
_abiArray=JSON.parse(_abi);      //JSON parsing needed!!

async function doIt() {

    var myBank = new web3.eth.Contract(_abiArray,"0x32D192b0b16505aB0188748cf427a4d17Fbc15B6");
    
    var event = myBank.events.PrintLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
        }
    });
    const accounts = await web3.eth.getAccounts();
    
    await myBank.methods.deposit(11111).send({from:accounts[0] , gas:255766, value: 11111});
    await myBank.methods.deposit(222).send({from:accounts[0] , gas:255766, value: 222});;
    
    // - 입금 11111 wei, 222 wei를 하고 컨트랙잔고 11333 wei 출력
    const balanceJson = await myBank.methods.getBalance().call();
    balance = JSON.stringify(balanceJson);
    console.log("Balance of contract: " + balance);
    
    // - 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력
    await myBank.methods.forwardTo(accounts[1]).send({from: accounts[0] , gas:255766, value: 333});
    
    const balanceJson2 = await myBank.methods.getBalance().call();
    const balance2 = JSON.stringify(balanceJson2);
    console.log("Balance of contract: " + balance2);
    
    balance = await web3.eth.getBalance(accounts[0]);
    console.log("Balance of account[0]: " + balance);
    
    balance = await web3.eth.getBalance(accounts[1]);
    console.log("Balance of account[1]: " + balance);
   
    // - 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)
    balancebefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before withdrawall: " + balancebefore);
    
    await myBank.methods.withdrawAll().send({from:accounts[0]});
  
    balanceafter = await web3.eth.getBalance(accounts[0]);
    
    subtract = balanceafter-balancebefore;
    console.log("Balance after withdrawal: " + balanceafter);
    console.log("BalanceAfter - BalanceBefore: "+ subtract);
    
    //- 컨트랙으로 111 송금하고, deposit()이 아니라 fallback으로 입금하세요. 컨트랙잔고 111 wei 출력.
    await web3.eth.sendTransaction({from:accounts[0], to:"0x309DD201450E258AE229E3C47D9c8C91a70F0123", gas: 230000, value: 111});
    const balanceJson3 = await myBank.methods.getBalance().call();
    const balance3 = JSON.stringify(balanceJson3);
    console.log("Balance: " + balance3);
    process.exit(1); //force exit to disconnect websocket
}

doIt()

Overwriting src/BankUse.js


In [102]:
!node src/BankUse.js

Balance of contract: {"0":"11333","1":"11333"}
Balance of contract: {"0":"11000","1":"11333"}
Balance of account[0]: 999999040220548731226
Balance of account[1]: 1000000000000000000333
Balance before withdrawall: 999999040220548731226
Balance after withdrawal: 999998940325693503599
BalanceAfter - BalanceBefore: -99894855204864
Balance: {"0":"115792089237316195423570985008687907853269984665640564039457584007913129639603","1":"0"}
